In [1]:
!pip install python-docx langchain

!pip install -U langchain-community
!pip install torch
!pip install sentence_transformers
!pip install faiss-gpu
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 26.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [2]:
#import zipfile
import docx

from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    
)
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.vectorstores import FAISS

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.vectorstores import Chroma

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,pipeline

import torch
from torch import cuda
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline



from langchain.schema.runnable import RunnablePassthrough

from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS




2024-07-05 17:39:16.199690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 17:39:16.199839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 17:39:16.356692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [5]:
doc = getText("/kaggle/input/documentation/train_data_VK/documentation.docx")
doc[:100]

'Разработчикам\nДля управления приложениями предусмотрена система управления контентом –\xa0RuStore Консо'

In [6]:
splitter = CharacterTextSplitter(
    chunk_size=500,  
    chunk_overlap=100,  
    length_function=len,  
)

split_documents = splitter.create_documents([doc])

In [7]:
split_documents[0].page_content

'Разработчикам\nДля управления приложениями предусмотрена система управления контентом –\xa0RuStore Консоль.\nВход в систему осуществляется с помощью единого аккаунта VK ID.\nДоступ\xa0разработчика\xa0к системе RuStore Консоль\xa0осуществляется через web-интерфейс.\nАккаунт разработчика\nРегистрация разработчиков\nВы можете создать аккаунт юридического или физического лица в системе RuStore Консоль.\nРегистрация юридических лиц/индивидуальных предпринимателей предназначена для:\nкоммерческих и промышленных предприятий;\nгосударственных компаний;\nпрофессиональных разработчиков.\nПри выборе регистрации\xa0юридических лиц/индивидуальных предпринимателей необходимо подписать форму регистрации квалифицированной электронной подписью.\nРегистрация физических лиц предназначена для использования в личных целях. Аккаунт подойдет для:\nстудентов;\nлюбителей;\nполупрофессиональных разработчиков.\nПри выборе регистрации физических лиц необходимо пройти удаленную верификацию личности.'

In [8]:
len(split_documents)

419

In [9]:
split_documents[105]

Document(page_content='Информация о договоре\nИмя главного бухгалтера компании.\nБухгалтерская отчетность: отчет о прибылях и убытках, балансовый отчёт.\nДокументы, подтверждающие налоговое резидентство: свидетельство о налоговом резидентстве предоставляется во избежание двойного налогообложения. В дальнейшем мы попросим вас обновлять этот документ ежегодно.\n\nБанковские реквизиты\nУкажите реквизиты счёта компании в российском или зарубежном банке, по которым будут производиться расчёты.')

In [10]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)



/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_community.vectorstores import Chroma


vector_db = Chroma.from_documents(
    split_documents, 
    embeddings, # model to use for embedding the document chunks before storing.
    persist_directory='vector_db', # persist the database in memory.
    collection_name='ai_career' # name of the collection to store the chunks in.
)

In [12]:
query = "Как узнать VK ID?"

# return the chunks of the most similar five embeddings in the db
docs = vector_db.similarity_search(query, k=5)

docs

[Document(page_content='Если в вашем аккаунте перестала работать авторизация через Google, войдите заново другим способом: по номеру телефона, VK, Яндекс или ГосУслуги. Укажите тот же номер телефона, который использовали для авторизации ранее.\n\nКак узнать VK ID\nVK ID\xa0— это единый аккаунт для сервисов VK. Если у вас есть профиль в социальной сети ВКонтакте, он автоматически стал и вашим VK ID.'),
 Document(page_content='-\xa0Как узнать VK ID в RuStore Консоль?-\xa0Как войти с аккаунтом VK ID в мобильное приложение RuStore?\n\nОткройте ваше приложение на устройстве или установите его, если это еще не сделано.\nСовершите покупку в приложении. Например, оформите подписку. Во время оплаты появится окошко оплаты с отметкой «Тестовая покупка».'),
 Document(page_content='Выберите «Юридическое лицо или ИП».\n\n\nНажмите «Войти через VK ID».\n\nВведите номер телефона, привязанный к вашему аккаунту VK ID, или создайте новый аккаунт. Создание аккаунта доступно с использованием международных 

In [13]:
retriever = vector_db.as_retriever(
    search_type="similarity",  
    k=4,  
    score_threshold=None,)

In [14]:
retriever.get_relevant_documents("Как узнать VK ID?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='Если в вашем аккаунте перестала работать авторизация через Google, войдите заново другим способом: по номеру телефона, VK, Яндекс или ГосУслуги. Укажите тот же номер телефона, который использовали для авторизации ранее.\n\nКак узнать VK ID\nVK ID\xa0— это единый аккаунт для сервисов VK. Если у вас есть профиль в социальной сети ВКонтакте, он автоматически стал и вашим VK ID.'),
 Document(page_content='-\xa0Как узнать VK ID в RuStore Консоль?-\xa0Как войти с аккаунтом VK ID в мобильное приложение RuStore?\n\nОткройте ваше приложение на устройстве или установите его, если это еще не сделано.\nСовершите покупку в приложении. Например, оформите подписку. Во время оплаты появится окошко оплаты с отметкой «Тестовая покупка».'),
 Document(page_content='Выберите «Юридическое лицо или ИП».\n\n\nНажмите «Войти через VK ID».\n\nВведите номер телефона, привязанный к вашему аккаунту VK ID, или создайте новый аккаунт. Создание аккаунта доступно с использованием международных 

In [15]:
model_id = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

model = AutoModelForCausalLM.from_pretrained(model_id,do_sample=True,device_map = "auto")

/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 637539072 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [17]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [18]:
prompt_template = """
Instruction: Ответьте на вопрос, основываясь на следующем контексте:
{context}

Question:
{question} 
 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [19]:
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [20]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


rag_chain = ( 
 {"context": retriever| format_docs, "question": RunnablePassthrough()}
    | llm_chain
)

In [21]:
response = rag_chain.invoke("Как узнать VK ID?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [22]:
print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))

Question: Как узнать VK ID?

Instruction: Ответьте на вопрос, основываясь на следующем контексте:
Если в вашем аккаунте перестала работать авторизация через Google, войдите заново другим способом: по номеру телефона, VK, Яндекс или ГосУслуги. Укажите тот же номер телефона, который использовали для авторизации ранее.

Как узнать VK ID
VK ID — это единый аккаунт для сервисов VK. Если у вас есть профиль в социальной сети ВКонтакте, он автоматически стал и вашим VK ID.

- Как узнать VK ID в RuStore Консоль?- Как войти с аккаунтом VK ID в мобильное приложение RuStore?

Откройте ваше приложение на устройстве или установите его, если это еще не сделано.
Совершите покупку в приложении. Например, оформите подписку. Во время оплаты появится окошко оплаты с отметкой «Тестовая покупка».

Выберите «Юридическое лицо или ИП».


Нажмите «Войти через VK ID».

Введите номер телефона, привязанный к вашему аккаунту VK ID, или создайте новый аккаунт. Создание аккаунта доступно с использованием международны